In [1]:
import pandas as pd           # data mnipulation
import numpy as np            # number manipulation/crunching
import matplotlib.pyplot as plt  # plotting
from sklearn.metrics import  accuracy_score, classification_report 
import openpyxl
import sklearn
# Train Test split
from sklearn.model_selection import train_test_split
# Random forest classifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from anchor import utils
from anchor import anchor_tabular
from sklearn.model_selection import LeaveOneOut
import lime
import lime.lime_tabular
from collections import Counter
import shap
import math

abalone = pd.read_csv("abalone.csv")
Y = abalone['Outcome']
target_names=["0", "1"]
       
X = abalone[[ 'Length',	'Diameter',	'Height',	'Whole_weight',	'Shucked_weight',	'Viscera_weight',	'Shell_weight',	'Rings'  ]]
maximums = np.amax(np.array(X), axis=0)
#X = diabetes.iloc[:,:-1].values
X_featurenames = X.columns

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)
counter = Counter(Y)
model = sklearn.svm.SVC(kernel='rbf', probability=True)
model.fit(X_train, Y_train)
pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, pred)
print("Accuracy:", accuracy)
print(classification_report(Y_test, pred, target_names=["0", "1"]))

# model = KNeighborsClassifier(n_neighbors=11)
# model.fit(X_train, Y_train)
# predicted_knn = model.predict(X_test)
# accuracy_knn = accuracy_score(Y_test, predicted_knn)
# print("Precisión Knn ", accuracy_knn)

shap.initjs()

dataframe_columns_Shap = []
test_index = "test_index"
dataframe_columns_Shap.append(test_index)
for feature in X_featurenames:
    feature_shap_value = feature + "_shap_value"
    feature_sign = feature + "_sign"
    feature_rank = feature + "_rank"
    dataframe_columns_Shap.append(feature_shap_value)
    dataframe_columns_Shap.append(feature_sign)
    dataframe_columns_Shap.append(feature_rank)
expected_value_0 = "expected_value_0"
expected_value_1 = "expected_value_1"
f_0 = "f_0"
f_1 = "f_1"
probability_male = "probability_male"
probability_female = "probability_female"
accuracy_classifier = "accuracy_classifier"
true_class = "true_class"
dataframe_columns_Shap.append(expected_value_0)
dataframe_columns_Shap.append(expected_value_1)
dataframe_columns_Shap.append(f_0)
dataframe_columns_Shap.append(f_1)
dataframe_columns_Shap.append(probability_male)
dataframe_columns_Shap.append(probability_female)
dataframe_columns_Shap.append(accuracy_classifier)
dataframe_columns_Shap.append(true_class)
df_Shap = pd.DataFrame(columns = dataframe_columns_Shap)


cv = LeaveOneOut()
number_iterations = 1

for train_ix, test_ix in cv.split(X):
    print(test_ix[0])
    for count in range(number_iterations):
        row_Shap = []
        row_Shap.append(test_ix[0])
        model = sklearn.svm.SVC(kernel='rbf', probability=True)
        X_train, X_test = X.to_numpy()[train_ix, :], X.to_numpy()[test_ix, :]
        Y_train, Y_test = Y[train_ix], Y[test_ix]
        model.fit(X_train, Y_train)
        # a summarized version of the dataset using shap.kmeans. This has been done to reduce computation time
        explainer = shap.KernelExplainer(model.predict_proba, shap.kmeans(X_train, 20))
        #explainer = shap.KernelExplainer(model.predict_proba, X_train)
        shap_values = explainer.shap_values(X_test)
        #print(shap_values)
        shap_values = np.delete(shap_values, obj = 1, axis = 0).flatten()
        #print(shap_values)
        predict_proba = model.predict_proba(X_test).flatten()
        #print(predict_proba)
        ranks = []
        for i in range(len(X_featurenames)):
            count_lower = 0
            for j in range(len(X_featurenames)):
                if abs(shap_values[j]) < abs(shap_values[i]):
                    count_lower = count_lower + 1
            position = len(X_featurenames) - count_lower
            row_Shap.append(shap_values[i])
            if shap_values[i] > 0 :
                sign = "positive"
            else :
                sign = "negative"
            row_Shap.append(sign)
            row_Shap.append(position)
            ranks.append(position)
        #print(ranks)
        #print(X_featurenames)
        #print(explainer.expected_value)
        row_Shap.append(explainer.expected_value[0])
        row_Shap.append(explainer.expected_value[1])
        sum_shap_values = np.array(shap_values).sum(0) 
        f_0 = explainer.expected_value[0] + sum_shap_values
        f_1 = explainer.expected_value[1] - sum_shap_values
        #print(f_0)
        #print(f_1)
        row_Shap.append(f_0)
        row_Shap.append(f_1)
        probability_no_male = predict_proba[0]
        probability_female = predict_proba[1]
        row_Shap.append(probability_male)
        row_Shap.append(probability_female)
        pred = model.predict(X_test)
        accuracy_classifier = accuracy_score(Y_test, pred)
        true_class = target_names[Y_test[test_ix[0]]]
        row_Shap.append(accuracy_classifier)
        row_Shap.append(true_class)
        print(row_Shap)
        df_Shap.loc[test_ix[0] * number_iterations + count] = row_Shap
        
df_Shap.to_excel("outputAbaloneKernelShapLOOCV.xlsx")

Accuracy: 0.5502645502645502
              precision    recall  f1-score   support

           0       0.55      0.96      0.70       313
           1       0.48      0.04      0.07       254

    accuracy                           0.55       567
   macro avg       0.51      0.50      0.39       567
weighted avg       0.52      0.55      0.42       567



0


  0%|          | 0/1 [00:00<?, ?it/s]

[0, 0.00010013245115202921, 'positive', 5, 9.092499723104037e-05, 'positive', 6, 5.4973307055196946e-05, 'positive', 7, -0.00014109436423965347, 'negative', 3, -0.0004762866464274516, 'negative', 2, 0.00013575966483725448, 'positive', 4, 4.3988132256493686e-05, 'positive', 8, -0.004900463019104192, 'negative', 1, 0.5384788939074553, 0.4615211060925447, 0.533386828430216, 0.466613171569784, 'probability_male', 0.4666131715697841, 1.0, '0']
1


  0%|          | 0/1 [00:00<?, ?it/s]

...

[2832, -8.662365294738798e-06, 'negative', 7, -1.090485222762106e-05, 'negative', 6, -3.1889441825648413e-05, 'negative', 4, -2.9809719956767538e-05, 'negative', 5, 8.223502591667682e-05, 'positive', 2, -3.588107034155761e-05, 'negative', 3, -4.562931114056613e-06, 'negative', 8, 0.0013909552961436289, 'positive', 1, 0.5397245793476861, 0.46027542065231397, 0.541076059288986, 0.45892394071101406, 'probability_male', 0.4589239407110139, 1.0, '0']
2833


  0%|          | 0/1 [00:00<?, ?it/s]

[2833, 8.684695663719284e-06, 'positive', 4, 7.846241047522387e-06, 'positive', 5, -1.1259094844506882e-06, 'negative', 7, 5.864389407345114e-06, 'positive', 6, -4.123606957009596e-05, 'negative', 2, 9.050230014462089e-06, 'positive', 3, 6.781285947813582e-07, 'positive', 8, -0.0006248016762590659, 'negative', 1, 0.538943067419984, 0.46105693258001607, 0.5383080274493982, 0.46169197255060185, 'probability_male', 0.4616919725506018, 0.0, '1']
2834


  0%|          | 0/1 [00:00<?, ?it/s]

[2834, -6.28290065192936e-05, 'negative', 5, -6.267581125356078e-05, 'negative', 6, -2.4280946997132385e-05, 'negative', 8, -0.00012009875031134788, 'negative', 3, 0.0005629243087386237, 'positive', 2, -9.625498021939654e-05, 'negative', 4, -4.7100184259856586e-05, 'negative', 7, 0.0012402249068542053, 'positive', 1, 0.5390308603466473, 0.4609691396533529, 0.5404207698826795, 0.45957923011732066, 'probability_male', 0.4595792301173206, 1.0, '0']
